# Setup

In [1]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [2]:
# Clone `CreateDebateScraper` library from github
!git clone https://github.com/utkarsh512/CreateDebateScraper.git
%cd CreateDebateScraper/src/nested/

Cloning into 'CreateDebateScraper'...
remote: Enumerating objects: 176, done.
remote: Counting objects: 100% (6/6), done.
remote: Compressing objects: 100% (2/2), done.
remote: Total 176 (delta 5), reused 4 (delta 4), pack-reused 170
Receiving objects: 100% (176/176), 207.95 KiB | 4.62 MiB/s, done.
Resolving deltas: 100% (61/61), done.
/content/CreateDebateScraper/src/nested


In [23]:
!pip install cpnet

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
!pip install shifterator

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 40.4 MB 1.4 MB/s 


In [26]:
from   copy                     import deepcopy
import cpnet
from   itertools                import accumulate
import json
from   matplotlib               import pyplot as plt
import networkx as nx
import nltk
import numpy as np
import pandas as pd
import pickle
import re
from   scipy                    import stats
import textwrap
from   thread                   import Comment, Thread
from   tqdm                     import tqdm
nltk.download('punkt') # For tokenizers
nltk.download('stopwords')
import matplotlib
from   nltk.tokenize            import TweetTokenizer
from   nltk.corpus              import stopwords
from   pprint                   import pprint
import shifterator as sh
import wordcloud
# import skbio
matplotlib.rcParams.update({'font.size': 18})
matplotlib.rcParams["figure.figsize"] = (12, 5)
STOP_WORDS = list(stopwords.words('english'))

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [5]:
tknz = TweetTokenizer()

def clean_text(text):
    """
    Preprocessing text
    """
    text = text.lower()
    text = re.sub(r"http\S+", "", text)
    text = re.sub(r"www\S+", "", text)
    text = re.sub("-", " ", text)
    text = re.sub("\s+", " ", text)
    text = re.sub("\u2018", "X", text) 
    text = re.sub("\u2019", "X", text) 
    text = re.sub("\'", "X", text) 
    wordTokens_ = tknz.tokenize(text)
    wordTokens = list()
    for x in wordTokens_:
        x = ''.join([v for v in x if v.isalnum() or v == ' '])
        if len(x) > 0 and x != 'X':
            x = x.replace('X', '\'')
            wordTokens.append(x)
    return wordTokens

In [6]:
comments = dict()

# Topical forums on CreateDebate. We have scraped comments for all of the
# following forurm.
categories = ['business', 'comedy', 'entertainment', 'health', 'law', 'nsfw',
              'politics2', 'religion', 'science', 'shopping', 'sports',
              'technology', 'travel', 'world']

# However, we will be analyzing comments from selected forum only!
# These forum have at least 10k comments each.
categories_selected = ['politics2', 'religion', 'world', 
                       'science', 'law', 'technology']

for x in categories_selected:
    comments[x] = list()

In [7]:
# Loading comments from select forums

for cat in tqdm(categories_selected):
    fp = open('/content/gdrive/MyDrive/DL/CreateDebate/' + cat + '/threads.log', 'rb')

    # Get all the `Thread` objects pickled while scraping.
    threads = list()
    try:
        while True:
            e = pickle.load(fp)
            threads.append(e)
    except EOFError:
        fp.close()

    # While classifying CreateDebate comments, we used comments as per author mode.
    # Hence, using the same mode to attach classification score with the comments.
    # 
    # score < 0.5 -> ad hominem comment
    #       > 0.5 -> non ad hominem comment
    authors = dict()
    for thread in threads:
        for k, v in thread.comments.items():
            try:
                authors[v.author].append(v)
            except:
                authors[v.author] = list()
                authors[v.author].append(v)

    ctr = 0
    # Load the classification score of the comments.
    with open('/content/gdrive/MyDrive/DL/CreateDebate/' + cat + '/comments_with_score.log', 'rb') as fp:
        cws = pickle.load(fp)
    # Attach classification score with the comments.
    for author in authors.keys():
        for i in range(len(authors[author])):
            comment = authors[author][i]
            foo = deepcopy(comment.__dict__)
            foo['tag'] = cat
            foo['score'] = cws[ctr][0]
            foo['validation'] = cws[ctr][1][0]
            comments[cat].append(foo)
            ctr += 1

100%|██████████| 6/6 [00:19<00:00,  3.24s/it]


In [8]:
def parse_tstring(tstring):
    """
    Parses comment's time to an integer to enable
    comparison between comments based on their time of posting
    """
    if tstring == 'Not Available':
        raise ValueError('Invalid posting time for parse_tstring')
    tstring = tstring.replace('T', '-').replace(':', '-').replace('+', '-').split('-')
    return int(''.join(tstring[:-2]))

In [9]:
# Loading AH score

with open('/content/gdrive/MyDrive/Temp/47-ah-score.pkl', 'rb') as fp:
    ah_score = pickle.load(fp)

# `ah_score` is a dictionary that contains the ah score of the comments written
# by all the users

# key: category -> user
# value: list of ah_score for given user for given category

# value > 0.5 --> ad hominem
# value < 0.5 --> non ad hominem

In [10]:
# Loading CreateDebate profile characteristics into dataframe
df = pd.read_json('/content/gdrive/MyDrive/DL/CreateDebate/profile/results.json', lines=True)

# Extract useful characteristics
reward_points_map = {k : v for k, v in zip(df['username'].tolist(), df['reward_points'].tolist())}
efficiency_map    = {k : v for k, v in zip(df['username'].tolist(), df['efficiency'].tolist())}
allies_map        = {k : len(v) for k, v in zip(df['username'].tolist(), df['allies'].tolist())}
enemies_map       = {k : len(v) for k, v in zip(df['username'].tolist(), df['enemies'].tolist())}
hostiles_map      = {k : len(v) for k, v in zip(df['username'].tolist(), df['hostiles'].tolist())}

In [11]:
def profile_characteristics_stats(user_subset):
    """
    Returns average and standard deviation of characteristics for given subset
    of users
    """
    rewards_ = list()
    efficiency_ = list()
    n_allies = list()
    n_enemies = list()
    n_hostiles = list()

    for user in user_subset:
        try:
            rewards_.append(reward_points_map[user])
        except:pass
        try:
            efficiency_.append(efficiency_map[user])
        except:pass
        try:
            n_allies.append(allies_map[user])
        except:pass
        try:
            n_enemies.append(enemies_map[user])
        except:pass
        try:
            n_hostiles.append(hostiles_map[user])
        except:pass
    
    grpd_data = [rewards_, efficiency_, n_allies, n_enemies, n_hostiles]
    avgs = [np.average(x) for x in grpd_data]
    stds = [np.std(x) for x in grpd_data]
    
    return avgs, stds

In [12]:
# Maximum ah score per category per author
#   key: category -> author
#   value: maximum ah score

ah_score_max = dict()

for category, author_data in ah_score.items():
    ah_score_max[category] = dict()
    for author, ah_scores in author_data.items():
        ah_score_max[category][author] = np.max(ah_scores)

In [13]:
comment_count = dict()
# key: category -> author
# value: number of comments written by author in the given forum

for category in categories_selected:
    comment_count[category] = dict()

    for comment in comments[category]:
        author = comment['author']
        try:
            comment_count[category][author] += 1
        except KeyError:
            comment_count[category][author] = 1

In [14]:
user_list = set()

for category in categories_selected:
    for comment in comments[category]:
        user_list.add(comment['author'])

user_list = list(user_list)

In [15]:
first_post_time = dict()
# key: category -> user
# value: post time of the first comment by given user in the given category
#        It is an integer as returned by parse_tstring routine

for category in categories_selected:
    first_post_time[category] = dict()

    for comment in comments[category]: 
        if comment['time'] == 'Not Available':
            continue
        author = comment['author']
        try:
            first_post_time[category][author] = min(first_post_time[category][author], parse_tstring(comment['time']))
        except KeyError:
            first_post_time[category][author] = parse_tstring(comment['time'])

In [16]:
def get_migrated_users(categories_1, categories_2, categories_1_origin=True, require_migration=True):
    """
    Returns a list of usernames who migrated from categories_1 to categories_2

    If categories_1_origin is True, then only those users will be considered for whom the first post 
    in CreateDebate belongs to categories_1

    If require_migration
    """

    resultant_list = list()

    for user in user_list:
        post_time_1 = 20220101000000
        post_time_2 = 20220101000000

        if not isinstance(categories_1, set):
            categories_1 = set(categories_1)
        if not isinstance(categories_2, set): 
            categories_2 = set(categories_2)
        
        for category in categories_1:
            try:
                cur_post_time = first_post_time[category][user]
                post_time_1 = min(post_time_1, cur_post_time)
            except KeyError:
                pass
        
        for category in categories_2:
            try:
                cur_post_time = first_post_time[category][user]
                post_time_2 = min(post_time_2, cur_post_time) 
            except KeyError:
                pass

        if post_time_1 == 20220101000000 or post_time_2 == 20220101000000:
            continue

        if categories_1_origin:
            for category in categories_selected:
                if not ((category in categories_1) or (category in categories_2)):
                    try:
                        cur_post_time = first_post_time[category][user]
                        post_time_2 = min(post_time_2, cur_post_time)
                    except KeyError:
                        pass

        if post_time_1 < post_time_2 or not require_migration:
            resultant_list.append(user)
        
    return resultant_list

In [17]:
def partition_migrated_users(migration_list, categories_1, categories_2):
    """
    Partitions the users into 4 categories: 
        AH-AH
        AH-NonAH
        NonAH-AH
        NonAH-NonAH
    
    migration_list should be obtained using get_migrated_users method
    """

    ah_ah_list = []
    ah_nonah_list = []
    nonah_ah_list = []
    nonah_nonah_list = []

    for user in migration_list:
        max_score_1 = 0
        max_score_2 = 0
        for category in categories_1:
            max_score_1 = max(max_score_1, ah_score_max[category].get(user, 0))
        for category in categories_2:
            max_score_2 = max(max_score_2, ah_score_max[category].get(user, 0))

        if max_score_1 > 0.5 and max_score_2 > 0.5:
            ah_ah_list.append(user)

        elif max_score_1 > 0.5 and max_score_2 < 0.5:
            ah_nonah_list.append(user)
        
        elif max_score_1 < 0.5 and max_score_2 > 0.5:
            nonah_ah_list.append(user)

        elif max_score_1 < 0.5 and max_score_2 < 0.5:
            nonah_nonah_list.append(user)
        
        else:
            print(user)

    return ah_ah_list, ah_nonah_list, nonah_ah_list, nonah_nonah_list 

In [18]:
threads = []

for category in categories_selected:
    reader_addr = f'/content/gdrive/MyDrive/DL/CreateDebate/{category}/threads.log'
    reader = open(reader_addr, 'rb')
    try:
        while True:
            e = pickle.load(reader)
            threads.append(e)
    except:
        reader.close()

In [19]:
def build_graph(user_subset, n1 = 0, n2 = 0):
    """Builds support graph and dispute graph from hyper-parameters n1 and n2
    inputs
    :param n1: threshold on number of level-1 comments
    :param n2: threshold on number of direct replies

    output
    (author_map : dict, reverse_map : list, author_count : int, support_graph : nx.DiGraph, support_matrix: list, dispute_graph : nxDiGraph, dispute_matrix : list)
    """

    athr = dict()
    for e in threads:
        if 'root' in e.metaL.keys():
            for key in e.metaL['root'].keys():
                cmnt = e.comments[key]
                cur_athr = cmnt.author
                try:
                    athr[cur_athr] += 1
                except:
                    athr[cur_athr] = 1
        if 'root' in e.metaR.keys():
            for key in e.metaR['root'].keys():
                cmnt = e.comments[key]
                cur_athr = cmnt.author
                try:
                    athr[cur_athr] += 1
                except:
                    athr[cur_athr] = 1

    L1_athr = dict()
    for x in athr:
        if athr[x] >= n1:
            L1_athr[x] = True

    athr = dict()

    def dfs(Map, cmntMap, athr, cid='root'):
        if cid == 'root':
            for key in Map[cid].keys():
                dfs(Map[cid], cmntMap, athr, key)
            return
        cur_author = cmntMap[cid].author

        try:
            athr[cur_author] += len(Map[cid].keys())
        except:
            athr[cur_author] = len(Map[cid].keys())

        for key in Map[cid].keys():
            dfs(Map[cid], cmntMap, athr, key)

    for e in threads:
        if 'root' in e.metaL.keys():
            dfs(e.metaL, e.comments, athr)
        if 'root' in e.metaR.keys():
            dfs(e.metaR, e.comments, athr) 

    A = []
    for x in athr:
        if x not in user_subset:
            continue
        if athr[x] >= n2:
            try:
                z = L1_athr[x]
                A.append(x)
            except KeyError:
                pass

    author_map = dict()
    reverse_map = [""] * len(A)
    author_count = len(A)

    for i in range(author_count):
        author_map[A[i]] = i
        reverse_map[i] = A[i]

    support_matrix = [[0 for j in range(author_count)] for i in range(author_count)]
    dispute_matrix = [[0 for j in range(author_count)] for i in range(author_count)]

    def dfs1(Map, cmntMap, cid='root'):
        if cid == 'root':
            for key in Map[cid].keys():
                dfs1(Map[cid], cmntMap, key)
            return

        cur_author = cmntMap[cid].author
        cur_pol = cmntMap[cid].polarity
        
        if cur_author in author_map and cur_pol != 'Not Available':
            cur_author_id = author_map[cur_author]
            for key in Map[cid].keys():
                nxt_author = cmntMap[key].author
                nxt_pol = cmntMap[key].polarity
                if nxt_author in author_map and nxt_pol != 'Not Available':
                    nxt_author_id = author_map[nxt_author]
                    if cur_pol == nxt_pol:
                        support_matrix[nxt_author_id][cur_author_id] += 1
                    else:
                        dispute_matrix[nxt_author_id][cur_author_id] += 1

        for key in Map[cid].keys():
            dfs1(Map[cid], cmntMap, key)

    for e in threads:
        if 'root' in e.metaL:
            dfs1(e.metaL, e.comments)
        if 'root' in e.metaR:
            dfs1(e.metaR, e.comments)

    support_graph = nx.DiGraph()
    for i in range(author_count):
        for j in range(author_count):
            if support_matrix[i][j] != 0:
                support_graph.add_weighted_edges_from([(i, j, support_matrix[i][j])])

    dispute_graph = nx.DiGraph()
    for i in range(author_count):
        for j in range(author_count):
            if dispute_matrix[i][j] != 0:
                dispute_graph.add_weighted_edges_from([(i, j, dispute_matrix[i][j])])
    
    return (author_map, reverse_map, author_count, support_graph, support_matrix, dispute_graph, dispute_matrix)

In [135]:
def core_pheriphery(G):
    # https://github.com/skojaku/core-periphery-detection
    algorithm = cpnet.MINRES()
    algorithm.detect(G)
    pair_id = algorithm.get_pair_id()
    coreness = algorithm.get_coreness()
    return pair_id, coreness

In [20]:
user_map, user_reverse_map, user_count, support_graph, support_matrix, dispute_graph, dispute_matrix = build_graph(user_list)

In [21]:
support_centrality_dict = nx.algorithms.centrality.degree_centrality(support_graph)
dispute_centrality_dict = nx.algorithms.centrality.degree_centrality(dispute_graph)

In [22]:
support_clustering_dict = nx.algorithms.cluster.clustering(support_graph)
dispute_clustering_dict = nx.algorithms.cluster.clustering(dispute_graph)

In [142]:
support_pair_id, support_coreness = core_pheriphery(support_graph)
dispute_pair_id, dispute_coreness = core_pheriphery(dispute_graph)

In [143]:
def get_core_pheriphery(user_subset):
    s_groups = []
    s_coreness = []
    d_groups = []
    d_coreness = []

    for user in user_subset:
        try:
            s_groups.append(support_pair_id[user_map[user]])
        except:
            pass
        try:
            d_groups.append(dispute_pair_id[user_map[user]])
        except:
            pass
        try:
            s_coreness.append(support_coreness[user_map[user]])
        except:
            pass
        try:
            d_coreness.append(dispute_coreness[user_map[user]])
        except:
            pass
    
    return s_groups, s_coreness, d_groups, d_coreness

In [144]:
def get_clustering_stats(user_subset):
    s_c = []
    d_c = []

    for user in user_subset:
        try:
            s_c.append(support_clustering_dict[user_map[user]])
        except:
            pass
        try:
            d_c.append(dispute_clustering_dict[user_map[user]])
        except:
            pass
    
    return np.average(s_c), np.std(s_c), np.average(d_c), np.std(d_c)

In [145]:
def get_centrality_stats(user_subset):
    s_c = []
    d_c = []

    for user in user_subset:
        try:
            s_c.append(support_centrality_dict[user_map[user]])
        except:
            pass
        try:
            d_c.append(dispute_centrality_dict[user_map[user]])
        except:
            pass
    
    return np.average(s_c), np.std(s_c), np.average(d_c), np.std(d_c)

In [146]:
def get_reciprocity_stats(user_subset):
    _1, _2, _3, support_graph_, _4, dispute_graph_, _6 = build_graph(user_subset)

    try:
        support_graph_r = nx.algorithms.reciprocity(support_graph_)
    except:
        support_graph_r = 0

    try:
        dispute_graph_r = nx.algorithms.reciprocity(dispute_graph_)
    except:
        dispute_graph_r = 0

    return support_graph_r, dispute_graph_r

In [161]:
def plot_support_coreness(user_subset):
    _, s_coreness, *_ = get_core_pheriphery(user_subset)
    plt.hist(s_coreness, bins=[x / 100 for x in range(101)])
    plt.xlabel('Core-ness')
    plt.ylabel('#Users')
    plt.title('Core-ness distribution for support graph')

In [162]:
def plot_dispute_coreness(user_subset):
    _, _, _, d_coreness = get_core_pheriphery(user_subset)
    plt.hist(d_coreness, bins=[x / 100 for x in range(101)])
    plt.xlabel('Core-ness')
    plt.ylabel('#Users')
    plt.title('Core-ness distribution for dispute graph')

In [122]:
def get_core_pheriphery_stats(user_subset):
    s_groups, s_coreness, d_groups, d_coreness = get_core_pheriphery(user_subset)

    s_group_count = len(np.unique(s_groups))
    d_group_count = len(np.unique(d_groups))

    s_c_q = [0 for _ in range(4)]
    d_c_q = [0 for _ in range(4)]

    for x in s_coreness:
        if x < 0.25:
            s_c_q[0] += 1
        elif x >= 0.25 and x < 0.5:
            s_c_q[1] += 1
        elif x >= 0.5 and x < 0.75:
            s_c_q[2] += 1
        else:
            s_c_q[3] += 1
        
    for x in d_coreness:
        if x < 0.25:
            d_c_q[0] += 1
        elif x >= 0.25 and x < 0.5:
            d_c_q[1] += 1
        elif x >= 0.5 and x < 0.75:
            d_c_q[2] += 1
        else:
            d_c_q[3] += 1
    
    return s_group_count, d_group_count, s_c_q, d_c_q

In [52]:
def print_table_v2(user_subset):
    s_group_count, d_group_count, s_c_q, d_c_q = get_core_pheriphery_stats(user_subset)

    print('Support group count: %d' % s_group_count)
    print('Dispute group count: %d' % d_group_count)

    print(f'Support coreness quarantile: {s_c_q}')
    print(f'Dispute coreness quarantile: {d_c_q}')

In [43]:
def print_table(user_subset):
    n = len(user_subset)
    s_reci, d_reci = get_reciprocity_stats(user_subset) 
    s_deg_avg, s_deg_std, d_deg_avg, d_deg_std = get_centrality_stats(user_subset)
    s_clu_avg, s_clu_std, d_clu_avg, d_clu_std = get_clustering_stats(user_subset)
    user_chr_avg, user_chr_std = profile_characteristics_stats(user_subset) 

    print('Size: %d' % n)
    print('Support graph reciprocity: %.2f' % s_reci)
    print('Dispute graph reciprocity: %.2f' % d_reci)

    print('Support graph degree centrality: %.5f ± %.5f' % (s_deg_avg, s_deg_std))
    print('Dispute graph degree centrality: %.5f ± %.5f' % (d_deg_avg, d_deg_std)) 

    print('Support graph clustering coeff: %.2f ± %.2f' % (s_clu_avg, s_clu_std))
    print('Dispute graph clustering coeff: %.2f ± %.2f' % (d_clu_avg, d_clu_std)) 

    print('Reward points: %.2f ± %.2f' % (user_chr_avg[0], user_chr_std[0]))
    print('Efficiency   : %.2f ± %.2f' % (user_chr_avg[1], user_chr_std[1]))
    print('# Allies     : %.2f ± %.2f' % (user_chr_avg[2], user_chr_std[2]))
    print('# Enemies    : %.2f ± %.2f' % (user_chr_avg[3], user_chr_std[3]))
    print('# Hostiles   : %.2f ± %.2f' % (user_chr_avg[4], user_chr_std[4]))

# Analysis

In [176]:
categories_1 = ['religion']
categories_2 = ['politics2']

In [188]:
migration_list = get_migrated_users(categories_1, categories_2, categories_1_origin=False, require_migration=False)

In [189]:
len(migration_list)

1878

In [190]:
AA, AN, NA, NN = partition_migrated_users(migration_list, categories_1, categories_2)

In [191]:
print('AH-AH: %d, AH-NONAH: %d, NONAH-AH: %d, NONAH-NONAH: %d' % (len(AA), len(AN), len(NA), len(NN)))

AH-AH: 712, AH-NONAH: 270, NONAH-AH: 360, NONAH-NONAH: 536


In [192]:
A = AA + AN
N = NA + NN

In [103]:
print_table(AA)

Size: 712
Support graph reciprocity: 0.66
Dispute graph reciprocity: 0.72
Support graph degree centrality: 0.00839 ± 0.01279
Dispute graph degree centrality: 0.00895 ± 0.01300
Support graph clustering coeff: 0.30 ± 0.26
Dispute graph clustering coeff: 0.28 ± 0.22
Reward points: 1001.71 ± 2677.58
Efficiency   : 87.39 ± 9.65
# Allies     : 8.86 ± 21.52
# Enemies    : 3.39 ± 15.97
# Hostiles   : 2.87 ± 7.36


In [104]:
print_table(AN)

Size: 270
Support graph reciprocity: 0.45
Dispute graph reciprocity: 0.44
Support graph degree centrality: 0.00169 ± 0.00218
Dispute graph degree centrality: 0.00186 ± 0.00199
Support graph clustering coeff: 0.20 ± 0.27
Dispute graph clustering coeff: 0.22 ± 0.24
Reward points: 83.21 ± 121.95
Efficiency   : 88.75 ± 9.44
# Allies     : 2.06 ± 7.45
# Enemies    : 0.33 ± 1.05
# Hostiles   : 0.25 ± 0.73


In [105]:
print_table(NA)

Size: 360
Support graph reciprocity: 0.14
Dispute graph reciprocity: 0.33
Support graph degree centrality: 0.00143 ± 0.00182
Dispute graph degree centrality: 0.00170 ± 0.00206
Support graph clustering coeff: 0.24 ± 0.32
Dispute graph clustering coeff: 0.27 ± 0.30
Reward points: 93.43 ± 130.25
Efficiency   : 85.58 ± 11.51
# Allies     : 2.17 ± 5.79
# Enemies    : 0.73 ± 2.42
# Hostiles   : 0.59 ± 1.30


In [106]:
print_table(NN)

Size: 536
Support graph reciprocity: 0.11
Dispute graph reciprocity: 0.08
Support graph degree centrality: 0.00068 ± 0.00079
Dispute graph degree centrality: 0.00063 ± 0.00063
Support graph clustering coeff: 0.13 ± 0.28
Dispute graph clustering coeff: 0.15 ± 0.28
Reward points: 34.49 ± 58.58
Efficiency   : 87.28 ± 11.40
# Allies     : 0.89 ± 2.75
# Enemies    : 0.12 ± 0.47
# Hostiles   : 0.09 ± 0.39


In [107]:
print_table(A)

Size: 982
Support graph reciprocity: 0.65
Dispute graph reciprocity: 0.70
Support graph degree centrality: 0.00681 ± 0.01158
Dispute graph degree centrality: 0.00711 ± 0.01165
Support graph clustering coeff: 0.27 ± 0.26
Dispute graph clustering coeff: 0.26 ± 0.22
Reward points: 748.66 ± 2316.61
Efficiency   : 87.77 ± 9.61
# Allies     : 6.98 ± 18.98
# Enemies    : 2.55 ± 13.67
# Hostiles   : 2.15 ± 6.38


In [108]:
print_table(N)

Size: 896
Support graph reciprocity: 0.11
Dispute graph reciprocity: 0.26
Support graph degree centrality: 0.00106 ± 0.00145
Dispute graph degree centrality: 0.00113 ± 0.00157
Support graph clustering coeff: 0.19 ± 0.30
Dispute graph clustering coeff: 0.21 ± 0.30
Reward points: 58.20 ± 98.54
Efficiency   : 86.59 ± 11.48
# Allies     : 1.41 ± 4.29
# Enemies    : 0.37 ± 1.60
# Hostiles   : 0.29 ± 0.91


In [182]:
# Coreness 

print_table_v2(AA)

Support group count: 1
Dispute group count: 1
Support coreness quarantile: [128, 64, 21, 0]
Dispute coreness quarantile: [129, 65, 31, 0]


In [193]:
print_table_v2(AN)

Support group count: 1
Dispute group count: 1
Support coreness quarantile: [123, 55, 28, 0]
Dispute coreness quarantile: [152, 64, 26, 0]


In [194]:
print_table_v2(NA)

Support group count: 1
Dispute group count: 1
Support coreness quarantile: [172, 79, 23, 0]
Dispute coreness quarantile: [179, 99, 46, 0]


In [195]:
print_table_v2(NN)

Support group count: 1
Dispute group count: 1
Support coreness quarantile: [166, 66, 35, 0]
Dispute coreness quarantile: [249, 79, 40, 0]


In [196]:
print_table_v2(A)

Support group count: 1
Dispute group count: 1
Support coreness quarantile: [515, 253, 105, 0]
Dispute coreness quarantile: [550, 242, 136, 5]


In [197]:
print_table_v2(N)

Support group count: 1
Dispute group count: 1
Support coreness quarantile: [338, 145, 58, 0]
Dispute coreness quarantile: [428, 178, 86, 0]
